In [1]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import sqlite3
import os
import numpy as np
from requests_html import HTMLSession
from requests_file import FileAdapter
import datetime

anuehtml = 'https://news.cnyes.com/news/cat/tw_bull?exp=a'

session = HTMLSession()
session.mount('file://',FileAdapter())
pwd = os.getcwd().replace("\\","/")

# 使用 with 語句能夠自動關閉文件，更加簡潔和安全
with open(f'{pwd}/台股公告 _ Anue鉅亨.html','r', encoding='utf-8') as f:
    soup = BeautifulSoup(f, 'html.parser')

In [4]:
titletime = []
id = []
idBefore = []
content = []
  

# # 今日新闻
inputdate = '2023-05-11'


# # 日期============================
times = soup.select('._67tN')
for time in times:
  head,mid,tail = time.find('time')['datetime'].partition('T') # partition‘T'之後分為頭部，中部，尾部三個
  titletime.append(head)
# print(len(titletime))

# 內容===============================================================================================================
titles = soup.select('._1xc2')
# 1.去除特殊字符
for title in titles:
  head,mid,tail = title.find('h3').text.partition(':')
  if head[0] == '「':
    head2 = head.replace('「','') # replace（代替），將字符替換成
    head3 = head2.find('、') # 在第幾位

    if head3 < 0: # 沒有、號
      head2 = head2.split('」',1) # split（切割），切割為【」以前的內容】和【」以後的內容】
      head2 = head2[0]
      idBefore.append(head2)
    else: # 有、號
      head2 = head2[:head3]
      head2 = head2.split('」',1) 
      head2 = head2[0]
      idBefore.append(head2)
  else:
    idBefore.append(head)
    
  titleText = title.text
  titleText = titleText.strip() # 清除頭尾空格
  content.append(titleText)
  
# 2.去除【月營收】
filtermonth = datetime.datetime.today()
filtermonth = filtermonth.strftime("%m")

if filtermonth == "10" or filtermonth == "11" or filtermonth == "12":
# 如果月份>雙位數
  for x in idBefore:
    x2 = x.find('月營收')
    if x2 < 0: # 沒有【月營收】
      id.append(x)
    else: # 有【月營收】
      x3 = x2 - 2
      x4 = x[:x3]
      id.append(x4)
# 如果月份為當位數
else:
  for x in idBefore:
    x2 = x.find('月營收')
    if x2 < 0: # 沒有【月營收】
      id.append(x)
    else: # 有【月營收】
      x3 = x2 - 1
      x4 = x[:x3]
      id.append(x4)

# =================================================================================================================


data = {
  'date': titletime,
  'code': id,
  'content': content
}

df = pd.DataFrame(data)

# 去除特殊字符 ===================================================
for x in range(len(df['content'])):
  df.iloc[x]['content'] = df.iloc[x]['content'].replace('','')
  df.iloc[x]['content'] = df.iloc[x]['content'].replace('','')
  df.iloc[x]['content'] = df.iloc[x]['content'].replace('∕','')
  df.iloc[x]['content'] = df.iloc[x]['content'].replace('','')
# ===============================================================


newsToday = pd.Timestamp.today().strftime("%Y-%m-%d") # 格式化日期
df_mask = df['date']== inputdate            # 日期為今天的存檔
positions = np.flatnonzero(df_mask)
filtered_df=df.iloc[positions]
# print(filtered_df)


# connecting to sqlite
conn = sqlite3.connect('anuenews.db')
# cursor object
cursor = conn.cursor()
uniqueSql = '''select * from news t'''


for index, row in filtered_df.iterrows():
  try:
    cursor.execute(
        """INSERT OR IGNORE INTO news 
          (date, code, content)
          values(?,?,?)""",
          (row['date'],
          row['code'],
          row['content']),
        )
    conn.commit()
  except:
    pass

# print(filtered_df)

# 確認當日新聞 ========================================================================
dfToday = pd.read_sql(uniqueSql,conn)
# df_maskToday = dfToday['date']==newsToday            # 日期為今天的存檔
df_maskToday = dfToday['date']==inputdate           # 日期為今天的存檔
positionsToday = np.flatnonzero(df_maskToday)
filtered_dfToday=dfToday.iloc[positionsToday]
# ======================================================================================

filtered_dfToday

,date,code,content
42973,2023-05-11,仲琦,仲琦:仲琦科技股份有限公司（公司代號：2419）108年私募普通股股票上市掛牌日期。
42974,2023-05-11,帝寶,帝寶4月營收13.39億元年增2.37% 1—4月達59.15億元
42975,2023-05-11,達邦蛋白,達邦蛋白4月營收0.37億元年減32.52% 1—4月達1.36億元
42976,2023-05-11,晶心科,晶心科4月營收0.71億元年增56.29% 1—4月達2.96億元
42977,2023-05-11,明達醫,明達醫4月營收0.55億元年減25.9% 1—4月達2.63億元
...,...,...,...
43482,2023-05-11,興富發,興富發:本公司公告台中市建案停工說明
43483,2023-05-11,仲琦,仲琦:公告本公司108年度私募普通股補辦公開發&#12175;及上市相關事宜
43484,2023-05-11,宏亞,宏亞:宏亞食品股份有限公司發放112年現金股利公告
43485,2023-05-11,統一,統一4月營收440.70億元年增2.54% 1—4月達1754.73億元


# 導出CSV（CHATGPT)

In [3]:
import pandas as pd
import numpy as np
import sqlite3

# 使用 with 語句，可以自動關閉數據庫連接，更加簡潔和安全
with sqlite3.connect('anuenews.db') as conn2:
    uniqueSql2 = '''select * from news t'''
    df2 = pd.read_sql(uniqueSql2, conn2)
    
newsToday = '2023-04-20'
newsToday2 = newsToday.replace("-","")
filtered_df = df2[df2['date']==newsToday][['code','content','date']] # 使用鏈式操作

filtered_df.to_csv(f"csv/{newsToday2}鉅亨新聞.csv", encoding="big5hkscs",index=False)
print(f'{newsToday2}導出成功')

20230420導出成功


# 批量導出CSV

In [5]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import sqlite3

with sqlite3.connect('anuenews.db') as conn2:
    uniqueSql2 = '''select * from news t'''
    df2 = pd.read_sql(uniqueSql2, conn2)

# 【日期】==============================================
newsToday1 = '2023-05-09'                             #|
newsToday2 = '2023-05-11'                             #|
# =====================================================


# 指定日期格式進行轉換
date_format = "%Y-%m-%d"  # 字串日期的格式
newsToday1 = datetime.strptime(newsToday1, date_format)
newsToday2 = datetime.strptime(newsToday2, date_format)


# 創建空的DataFrame
dataExport1 = pd.DataFrame(columns = ['code','content','date'])


# 計算日期範圍
date_range = newsToday2 - newsToday1

# 遍歷日期範圍並印出每一天
for i in range(date_range.days + 1):
    current_date = newsToday1 + timedelta(days=i)
    current_date = str(current_date.strftime("%Y-%m-%d"))

    filtered_df = df2[df2['date']==current_date][['code','content','date']] # 使用鏈式操作
    dataExport1 = pd.concat([dataExport1,filtered_df],axis=0,ignore_index = True)
    
# 日期從新到舊
dataExport1 = dataExport1.sort_values(by='date', ascending=False)

newsToday3 = str(newsToday2.strftime("%Y-%m-%d")).replace("-","")
dataExport1.to_csv(f"csv/{newsToday3}鉅亨新聞.csv", encoding="big5hkscs",index=False)
print(f'{newsToday3}導出成功')
    


20230511導出成功


---

# 導出CSV舊版

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import sqlite3
import os
import numpy as np
from requests_html import HTMLSession
from requests_file import FileAdapter
import datetime

session = HTMLSession()

session.mount('file://',FileAdapter())
pwd = os.getcwd().replace("\\","/")
html_obj = session.get(f'file:///{pwd}/台股公告 _ Anue鉅亨.html')
soup = BeautifulSoup(html_obj.text,"html.parser")
html_obj.close()

# 使用 with 語句能夠自動關閉文件，更加簡潔和安全
# with open(f'{pwd}/台股公告 _ Anue鉅亨.html','r', encoding='utf-8') as f:
#     soup = BeautifulSoup(f, 'html.parser')

In [ ]:
id = []
idBefore = []
content = []
  

# 今日新闻
inputdate = '2023-03-23'


# 日期============================
times = [time.find('time')['datetime'].partition('T')[0] for time in soup.select('._67tN')] # partition‘T'之後分為頭部，中部，尾部三個
titles = [title.find('h3').text.strip() for title in soup.select('._1xc2')]

# 去除特殊字符 ===================================================
titles = [title.replace('','').replace('','').replace('∕','') for title in titles]
# ===============================================================

# 1.去除特殊字符
for title in titles:
  head,mid,tail = title.partition(':')
  head,mid,tail = head.partition('：')
  if head[0] == '「':
    head2 = head.replace('「','') # replace（代替），將字符替換成
    head3 = head2.find('、') # 在第幾位

    if head3 < 0: # 沒有、號
      head2 = head2.split('」',1) # split（切割），切割為【」以前的內容】和【」以後的內容】
      head2 = head2[0]
      idBefore.append(head2)
    else: # 有、號
      head2 = head2[:head3]
      head2 = head2.split('」',1) 
      head2 = head2[0]
      idBefore.append(head2)
  else:
    idBefore.append(head)
    
  content.append(title)
  
# 2.去除【月營收】
filtermonth = datetime.datetime.today()
filtermonth = filtermonth.strftime("%m")

if filtermonth == "10" or filtermonth == "11" or filtermonth == "12":
# 如果月份>雙位數
  for x in idBefore:
    x2 = x.find('月營收')
    if x2 < 0: # 沒有【月營收】
      id.append(x)
    else: # 有【月營收】
      x3 = x2 - 2
      x4 = x[:x3]
      id.append(x4)
# 如果月份為當位數
else:
  for x in idBefore:
    x2 = x.find('月營收')
    if x2 < 0: # 沒有【月營收】
      id.append(x)
    else: # 有【月營收】
      x3 = x2 - 1
      x4 = x[:x3]
      id.append(x4)

# =================================================================================================================

# 創建DataFrame
data = {
  'date': times,
  'code': id,
  'content': content
}
df = pd.DataFrame(data)

# 篩選今日新聞
filtered_df = df[df['date'] == inputdate]

# 去重並寫入數據庫
unique_Sql = '''select * from news t'''
with sqlite3.connect('anuenews.db') as conn:
  try:
    df_today = pd.read_sql(unique_Sql, conn)
    filtered_df_today = df_today[df_today['date'] == inputdate]
    filtered_df = filtered_df[~filtered_df.isin(filtered_df_today)].dropna()
    filtered_df.to_sql('news', conn, if_exists='append', index=False)
  except:
    pass
    
filtered_df_today

In [18]:
titletime = []
id = []
idBefore = []
content = []
  

# # 今日新闻
inputdate = '2023-03-23'


# # 日期============================
times = soup.select('._67tN')
for time in times:
  head,mid,tail = time.find('time')['datetime'].partition('T') # partition‘T'之後分為頭部，中部，尾部三個
  titletime.append(head)
# print(len(titletime))

# 內容===============================================================================================================
titles = soup.select('._1xc2')
# 1.去除特殊字符
for title in titles:
  head,mid,tail = title.find('h3').text.partition(':')
  if head[0] == '「':
    head2 = head.replace('「','') # replace（代替），將字符替換成
    head3 = head2.find('、') # 在第幾位

    if head3 < 0: # 沒有、號
      head2 = head2.split('」',1) # split（切割），切割為【」以前的內容】和【」以後的內容】
      head2 = head2[0]
      idBefore.append(head2)
    else: # 有、號
      head2 = head2[:head3]
      head2 = head2.split('」',1) 
      head2 = head2[0]
      idBefore.append(head2)
  else:
    idBefore.append(head)
    
  titleText = title.text
  titleText = titleText.strip() # 清除頭尾空格
  content.append(titleText)
  
# 2.去除【月營收】
filtermonth = datetime.datetime.today()
filtermonth = filtermonth.strftime("%m")

if filtermonth == "10" or filtermonth == "11" or filtermonth == "12":
# 如果月份>雙位數
  for x in idBefore:
    x2 = x.find('月營收')
    if x2 < 0: # 沒有【月營收】
      id.append(x)
    else: # 有【月營收】
      x3 = x2 - 2
      x4 = x[:x3]
      id.append(x4)
# 如果月份為當位數
else:
  for x in idBefore:
    x2 = x.find('月營收')
    if x2 < 0: # 沒有【月營收】
      id.append(x)
    else: # 有【月營收】
      x3 = x2 - 1
      x4 = x[:x3]
      id.append(x4)

# =================================================================================================================


data = {
  'date': titletime,
  'code': id,
  'content': content
}

df = pd.DataFrame(data)

# 去除特殊字符 ===================================================
for x in range(len(df['content'])):
  df.iloc[x]['content'] = df.iloc[x]['content'].replace('','')
  df.iloc[x]['content'] = df.iloc[x]['content'].replace('','')
  df.iloc[x]['content'] = df.iloc[x]['content'].replace('∕','')
# ===============================================================


newsToday = pd.Timestamp.today().strftime("%Y-%m-%d") # 格式化日期
df_mask = df['date']== inputdate            # 日期為今天的存檔
positions = np.flatnonzero(df_mask)
filtered_df=df.iloc[positions]
# print(filtered_df)


# connecting to sqlite
conn = sqlite3.connect('anuenews.db')
# cursor object
cursor = conn.cursor()
uniqueSql = '''select * from news t'''


for index, row in filtered_df.iterrows():
  try:
    cursor.execute(
        """INSERT OR IGNORE INTO news 
          (date, code, content)
          values(?,?,?)""",
          (row['date'],
          row['code'],
          row['content']),
        )
    conn.commit()
  except:
    pass

# print(filtered_df)

# 確認當日新聞 ========================================================================
dfToday = pd.read_sql(uniqueSql,conn)
# df_maskToday = dfToday['date']==newsToday            # 日期為今天的存檔
df_maskToday = dfToday['date']==inputdate           # 日期為今天的存檔
positionsToday = np.flatnonzero(df_maskToday)
filtered_dfToday=dfToday.iloc[positionsToday]
# ======================================================================================

filtered_dfToday

,date,code,content
32947,2023-03-23,迎輝,迎輝:公告本公司董事會通過子公司增資案
32948,2023-03-23,進泰電子,進泰電子:本公司董事會決議現金增資發行新股
32949,2023-03-23,進泰電子,進泰電子:本公司董事會決議召開112年股東常會相關事宜
32950,2023-03-23,邦泰,"邦泰:公告本公司董事會通過111年第1次現金增資私募案因股款未於繳款期間內收足新台幣170,..."
32951,2023-03-23,邦泰,邦泰:公告本公司自然人董事辭職
...,...,...,...
33492,2023-03-23,聯嘉,聯嘉:澄清經濟日報C3版112/03/23對本公司之報導
33493,2023-03-23,誠美材,「誠美材」現金減資換發新股於2023.04.24上市
33494,2023-03-23,誠品生活,誠品生活:澄清112年3月23日自由時報第A08版報導
33495,2023-03-23,晶心科,晶心科:澄清經濟日報報導


In [10]:
import pandas as pd
import numpy as np
import sqlite3


conn2 = sqlite3.connect('anuenews.db')
uniqueSql2 = '''select * from news t'''

df2 = pd.read_sql(uniqueSql2,conn2)
df3 = pd.DataFrame(df2, columns=['code','content','date'])

# 今日新闻
newsToday = '2023-03-14'
# newsToday = pd.Timestamp.today().strftime("%Y-%m-%d") # 格式化日期

df_mask = df3['date']==newsToday            # 日期為今天的存檔
positions = np.flatnonzero(df_mask)
filtered_df=df3.iloc[positions]

newsToday2 = newsToday.replace("-","")
# newsToday2 = pd.Timestamp.today().strftime("%Y%m%d")

filtered_df.to_csv(f"csv/{newsToday2}鉅亨新聞.csv", encoding="big5hkscs",index=False)

print(f'{newsToday2}導出成功')

20230314導出成功


In [18]:
import pandas as pd
import numpy as np
import sqlite3


conn2 = sqlite3.connect('anuenews.db')
uniqueSql2 = '''select * from news t'''

df2 = pd.read_sql(uniqueSql2,conn2)
df3 = pd.DataFrame(df2, columns=['code','content','date'])

# 今日新闻
newsToday = '2022-10-28'
# newsToday = pd.Timestamp.today().strftime("%Y-%m-%d") # 格式化日期

df_mask = df3['date']==newsToday            # 日期為今天的存檔
positions = np.flatnonzero(df_mask)
filtered_df=df3.iloc[positions]

newsToday2 = newsToday.replace("-","")
# newsToday2 = pd.Timestamp.today().strftime("%Y%m%d")

# filtered_df.to_csv(f"csv/{newsToday2}鉅亨新聞.csv", encoding="big5hkscs",index=False)

# print(f'{newsToday2}導出成功')
df3.iloc[0:55]

,code,content,date
0,廣華-KY,廣華-KY:公告本公司內部稽核主管異動,2022-08-27
1,廣華-KY,廣華-KY:董事會決議通過取消設置執行長乙職,2022-08-27
2,廣華-KY,廣華-KY:公告本公司經董事會通過之111年第二季自結合併財務報告,2022-08-27
3,聯德控股-KY,聯德控股-KY:公告本公司董事會決議分派111年第2季股利,2022-08-26
4,聯德控股-KY,聯德控股-KY:公告本公司董事會決議辦理庫藏股註銷暨訂定減資基準日,2022-08-26
5,聯德控股-KY,聯德控股-KY:公告本公司董事會通過111年第2季合併財務報告案,2022-08-26
6,菱光,菱光:公告本公司董事會決議撤銷111年第一次股東臨時會事宜,2022-08-26
7,訊芯-KY,"訊芯-KY:新增資金貸與子公司ShunSin Technology (Bag Giang,V...",2022-08-26
8,訊芯-KY,訊芯-KY:本公司暫停持股100%子公司訊芯香港增資事宜(更正110年12月29日重大訊息),2022-08-26
9,訊芯-KY,訊芯-KY:公告本公司董事會通過買回公司股份,2022-08-26


In [1]:
# 创建资料库 ====================================================================
# import sqlite3


# # connecting to sqlite
# conn = sqlite3.connect('anuenews.db')
# # cursor object
# cursor = conn.cursor()

# # Creating table
# news = """ CREATE TABLE news (
#     date DATE,
#     code TEXT,
#     content TEXT UNIQUE, 
#     UNIQUE(content)
# ); """

# cursor.execute(news)
# ================================================================================